In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, train_test_split, GridSearchCV

Считаем данные

In [2]:
CROC_PATH = 'data/crocodile/'
CLOCK_PATH = 'data/clock/'
DEBUG = False
croc_dict = dict()
clock_dict = dict()
croc_data = np.array([cv2.imread(CROC_PATH + i) for i in os.listdir(CROC_PATH)])
clock_data = np.array([cv2.imread(CLOCK_PATH + i) for i in os.listdir(CLOCK_PATH)])
'''
for i in os.listdir(CROC_PATH):
    if DEBUG: print('Reading image `{}`'.format(i))
    arr = cv2.imread(CROC_PATH + i)
    croc_dict[i.replace('.png', '')] = arr
    croc_data = np.append(croc_data, arr)
for i in os.listdir(CLOCK_PATH):
    arr = cv2.imread(CLOCK_PATH + i)
    clock_dict[i.replace('.png', '')] = arr
    np.append(croc_data, arr)
'''

"\nfor i in os.listdir(CROC_PATH):\n    if DEBUG: print('Reading image `{}`'.format(i))\n    arr = cv2.imread(CROC_PATH + i)\n    croc_dict[i.replace('.png', '')] = arr\n    croc_data = np.append(croc_data, arr)\nfor i in os.listdir(CLOCK_PATH):\n    arr = cv2.imread(CLOCK_PATH + i)\n    clock_dict[i.replace('.png', '')] = arr\n    np.append(croc_data, arr)\n"

Построим простые модели и оценим их качество.

### Линейная модель (можно пропустить при выполнении кода)

In [ ]:
ans = np.concatenate((np.ones(clock_data.shape[0]), np.zeros(croc_data.shape[0])))
data = np.concatenate((clock_data.reshape(clock_data.shape[0], -1), croc_data.reshape(croc_data.shape[0], -1)))
train_data, test_data, train_labels, test_labels = train_test_split(data, ans, test_size=0.3, random_state=0)
cv = StratifiedShuffleSplit(n_splits=10, random_state=0)

In [ ]:
reg = LogisticRegression(random_state=0)
param_grid = {'penalty': ['l1', 'l2'], 'C': [1, 10, 100]}
grid_cv = GridSearchCV(reg, param_grid, scoring='accuracy', cv=cv, n_jobs=4)
%time grid_cv.fit(train_data, train_labels)

In [ ]:
grid_cv.best_estimator_

In [ ]:
grid_cv.best_score_

Результаты не очень. Попробуем другие классификаторы:

In [ ]:
reg = RandomForestClassifier(random_state=0, n_jobs=4)
param_grid = {'n_estimators': [600],
              'max_depth': [30], 'criterion': ['entropy']}
grid_cv = GridSearchCV(reg, param_grid, scoring='accuracy', cv=cv, n_jobs=4)
%time grid_cv.fit(train_data, train_labels)

In [ ]:
grid_cv.best_estimator_

In [ ]:
grid_cv.best_score_

### Градиентный бустинг над случайными лесами (тоже можно пропустить)

In [ ]:
import xgboost as xgb

In [ ]:
reg = xgb.XGBClassifier(random_state=0, learning_rate=0.005, n_jobs=4, n_estimators=300)
#param_grid = {'n_estimators': [100, 200, 400, 600],
#              'learning_rate': [0.001, 0.005, 0.1]}
param_grid = {'n_estimators': [400],
              'learning_rate': [0.005]}
grid_cv = GridSearchCV(reg, param_grid, scoring='accuracy', cv=cv, n_jobs=4)
%time grid_cv.fit(train_data, train_labels)

In [ ]:
reg.fit(train_data, train_labels)

In [ ]:
cross_val_score(reg, test_data, test_labels)

## А теперь изюминка нашего проекта: CNN

In [3]:
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
ans = np.concatenate((np.ones(clock_data.shape[0]), np.zeros(croc_data.shape[0])))
data = np.concatenate((clock_data, croc_data))
train_data, test_data, train_labels, test_labels = train_test_split(data, ans, test_size=0.2, random_state=42)
#cv = StratifiedShuffleSplit(n_splits=10, random_state=0)

In [5]:
train_data.shape

(800, 32, 32, 3)

In [6]:
# Параметры модели. С ними надо будет поиграться
batch_size = 15
num_epochs = 28
kernel_size = 3
pool_size = 2
conv_depth_1 = 32
conv_depth_2 = 32
drop_prob_1 = 0.3
drop_prob_2 = 0.4
hidden_size = 128

In [7]:
num_train, depth, height, width = train_data.shape
num_test = test_data.shape[0]
num_classes = np.unique(train_labels).shape[0]

In [8]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= np.max(train_data)
test_data /= np.max(test_data)

In [9]:
Y_train = np_utils.to_categorical(train_labels, num_classes)
Y_test = np_utils.to_categorical(test_labels, num_classes)

In [10]:
inp = Input(shape=(depth, height, width))

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
#conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
#conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
drop_2 = Dropout(drop_prob_2)(pool_2)

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='tanh')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [11]:
model = Model(input=inp, output=out)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

In [17]:
model.fit(train_data, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)

Train on 720 samples, validate on 80 samples
Epoch 1/28
720/720 [==============================] - 2s 2ms/step - loss: 0.5713 - acc: 0.8764 - val_loss: 0.3036 - val_acc: 0.9000
Epoch 2/28
720/720 [==============================] - 1s 2ms/step - loss: 0.1294 - acc: 0.9444 - val_loss: 0.2382 - val_acc: 0.9125
Epoch 3/28
720/720 [==============================] - 1s 2ms/step - loss: 0.1199 - acc: 0.9542 - val_loss: 0.2476 - val_acc: 0.8875
Epoch 4/28
720/720 [==============================] - 1s 2ms/step - loss: 0.0872 - acc: 0.9681 - val_loss: 0.3475 - val_acc: 0.8375
Epoch 5/28
720/720 [==============================] - 1s 2ms/step - loss: 0.0936 - acc: 0.9639 - val_loss: 0.3223 - val_acc: 0.8625
Epoch 6/28
720/720 [==============================] - 1s 2ms/step - loss: 0.0731 - acc: 0.9708 - val_loss: 0.2767 - val_acc: 0.9125
Epoch 7/28
720/720 [==============================] - 1s 2ms/step - loss: 0.0574 - acc: 0.9792 - val_loss: 0.1969 - val_acc: 0.9125
Epoch 8/28
720/720 [===========

In [18]:
model.evaluate(test_data, Y_test, verbose=1)

200/200 [==============================] - 0s 828us/step


[0.4358933484554291, 0.9]